In [1]:
import numpy as np
from numpy import linalg
import cvxopt
import cvxopt.solvers
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, RegressorMixin
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
from sklearn import svm
import math 
import itertools

In [2]:
dataset=pd.read_csv("iris-setosa.csv")

X = dataset.columns[1:3]
X = dataset[X]
y = dataset.columns[0]
y = dataset[y]

X = np.array(X)
y = np.array(y)
X=X.astype(float)
y=y.astype(float)
y=np.where(y==0,-1,y)

In [3]:
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=3):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma):
    # print(-linalg.norm(x-y)**2)
    x=np.asarray(x)
    y=np.asarray(y)
    return np.exp((-linalg.norm(x-y)**2) / (2 * (sigma ** 2)))

In [16]:
from cvxopt import matrix
class HYP_SVM(BaseEstimator, RegressorMixin):
    
    def __init__(self, kernel=gaussian_kernel, sigma=None, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)
        self.sigma = sigma
        
    #geometric mean
    def gm(self, y_predict, y_test):
        print(y_predict)
        test_min=0
        test_max=0
        pred_min=0
        pred_max=0
        y_test=np.asarray(y_test)
        for i in range(0,len(y_test)):
            if(y_test[i]==1):
                test_min=test_min+1
            else:
                test_max=test_max+1
        #print("y_test min",test_min)       
        #print("y_test max",test_max)
        for i in range(0,len(y_predict)):
            if(y_predict[i]==1 and y_predict[i]==y_test[i]):
                pred_min=pred_min+1
            elif(y_predict[i]==-1 and y_predict[i]==y_test[i]):
                pred_max=pred_max+1
        #print("y_pred min",pred_min)       
        #print("y_pred max",pred_max)
        se=pred_min/test_min
        sp=pred_max/test_max
        #print(se,sp)
        gm=math.sqrt(se*sp)
        print("GM",gm) 
        return gm
        
    def score(self, X, y):
        y_predict=self.predict(X)
        gm=self.gm(y_predict, y)
        correct = np.sum(y_predict == y)
        print("%d out of %d predictions correct" % (correct, len(y_predict)))
        mse=mean_squared_error(y, y_predict)
        print("Accuracy",correct/len(y_predict))
        print("Errore quadratico medio: ", mse)
        return gm
            
          
    def get_params(self, deep=True):
        return {"C": self.C, "sigma": self.sigma}        
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self
    
    def m_func(self, X_train, y):
        n_samples, n_features = X_train.shape 
        self.K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                self.K[i,j] = gaussian_kernel(X_train[i], X_train[j], self.sigma)
               # print(K[i,j])
        X_train=np.asarray(X_train)
        K1 = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K1[i,j] = gaussian_kernel(X_train[i], X_train[j], self.sigma)
               # print(K[i,j])
        #print(K1.shape)
        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        #print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        a_org = np.ravel(solution['x'])
        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        #print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a_org=a
        self.a = a[sv]
        self.sv = X_train[sv]
        self.sv_y = y[sv]
        self.sv_yorg=y
        self.kernel = gaussian_kernel
        X_train=np.asarray(X_train)
        b = 0
        for n in range(len(self.a)):
            b += self.sv_y[n]
            b -= np.sum(self.a * self.sv_y * self.K[ind[n],sv])
        b /= len(self.a)
       # print(self.a_org[1])
        #print(self.a_org.shape,self.sv_yorg.shape,K.shape)
        w_phi=0
        total=0
        for n in range(len(self.a_org)):
            w_phi = self.a_org[n] * self.sv_yorg[n] * K1[n] 
        self.d_hyp=np.zeros(n_samples)
        for n in range(len(self.a_org)):
            self.d_hyp += self.sv_yorg[n]*(w_phi+b)
        func=np.zeros((n_samples))
        func=np.asarray(func)
        typ=1
        if(typ==1):
            for i in range(n_samples):
                func[i]=1-(self.d_hyp[i]/(np.amax(self.d_hyp[i])+0.000001))
        beta=0.8
        if(typ==2):
            for i in range(n_samples):
                func[i]=2/(1+beta*self.d_hyp[i])
        r_max=103/4074
        r_min=1
        self.m=func[0:115]*r_min
        #print(self.m.shape)
        self.m=np.append(self.m,func[115:5473]*r_max)
        #print(self.m.shape)
        
 ##############################################################################

    #prendeva come argomento anche x_test, l'ho tolto, ho aggiunto K
    def fit(self, X_train, y):
        
        self.m_func(X_train, y)
        self.kernel = gaussian_kernel
        n_samples, n_features = X_train.shape 

        # Gram matrix

        #print(self.K.shape)

        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        #print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        a_org = np.ravel(solution['x'])
        # Support vectors have non zero lagrange multipliers
        for i in range(n_samples):
            sv=np.logical_or(self.a_org <self.m, self.a_org > 1e-5)
        #print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X_train[sv]
        self.sv_y = y[sv]
        #print("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * self.K[ind[n],sv])
        self.b /= len(self.a)
        #print(self.b)

        # Weight vector
        if self.kernel == gaussian_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
        else :
            self.w = None
            
        return self    
        
    def project(self, X):
        if self.w is None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            X=np.asarray(X)
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * gaussian_kernel(X[i], sv, self.sigma)
                    #print(gaussian_kernel(X[i], sv, self.sigma)>0)
                y_predict[i] = s
                #Valori di membership
                #print(y_predict[i]+self.b)
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X)) #per l'accuracy
        #return self.project(X) per ritornare le membership

In [17]:
C_vals = [1e-2, 1e-1, 1, 1e+1, 1e+2, 1e+3, 1e+4]
sigma = [9e-2, 9e-1, 9, 9e+1, 9e+2, 9e+3, 9e+4]
#kernels = ["linear_kernel", "polynomial_kernel", "gaussian_kernel"]
parameters = {'C': C_vals, 'sigma': sigma}

model = HYP_SVM(C=100, kernel=gaussian_kernel, sigma=0.9)

xTrain, xTest, yTrain, yTest = train_test_split(X, y)

clf = GridSearchCV(model, parameters, cv=5)


In [18]:
grid_result = clf.fit(X=xTrain, y=yTrain)

     pcost       dcost       gap    pres   dres
 0: -1.5521e+01 -6.5038e+00  3e+02  2e+01  3e-16
 1: -5.6246e+00 -2.2894e+00  2e+01  1e+00  5e-16
 2: -6.5766e-01 -1.6810e+00  1e+00  6e-03  2e-15
 3: -6.8910e-01 -7.4651e-01  6e-02  3e-04  4e-16
 4: -6.9248e-01 -6.9717e-01  5e-03  2e-05  2e-16
 5: -6.9310e-01 -6.9409e-01  1e-03  2e-06  3e-16
 6: -6.9334e-01 -6.9348e-01  1e-04  1e-07  5e-16
 7: -6.9338e-01 -6.9340e-01  2e-05  1e-08  4e-16
 8: -6.9339e-01 -6.9339e-01  8e-07  2e-10  3e-16
 9: -6.9339e-01 -6.9339e-01  3e-08  3e-12  4e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -1.5521e+01 -6.5038e+00  3e+02  2e+01  3e-16
 1: -5.6246e+00 -2.2894e+00  2e+01  1e+00  5e-16
 2: -6.5766e-01 -1.6810e+00  1e+00  6e-03  2e-15
 3: -6.8910e-01 -7.4651e-01  6e-02  3e-04  4e-16
 4: -6.9248e-01 -6.9717e-01  5e-03  2e-05  2e-16
 5: -6.9310e-01 -6.9409e-01  1e-03  2e-06  3e-16
 6: -6.9334e-01 -6.9348e-01  1e-04  1e-07  5e-16
 7: -6.9338e-01 -6.9340e-01  2e-05  1e-08  4e-1

 -1. -1. -1. -1. -1.]
[ 1. -1. -1. -1.  1. -1. -1. -1.  1. -1. -1. -1.  1. -1. -1. -1.  1.  1.
 -1. -1. -1.  1. -1.]
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1.]
GM 0.0
16 out of 23 predictions correct
Accuracy 0.6956521739130435
Errore quadratico medio:  1.2173913043478262
     pcost       dcost       gap    pres   dres
 0: -5.9953e+00 -3.3130e+00  4e+02  2e+01  3e-16
 1: -2.3739e+00 -2.5560e+00  2e+01  1e+00  4e-16
 2: -6.6599e-01 -1.6948e+00  2e+00  6e-02  1e-15
 3: -5.3160e-01 -8.0762e-01  3e-01  2e-17  2e-15
 4: -5.4418e-01 -5.9714e-01  5e-02  2e-17  5e-16
 5: -5.5168e-01 -5.6334e-01  1e-02  3e-17  4e-16
 6: -5.5467e-01 -5.5810e-01  3e-03  2e-17  4e-16
 7: -5.5560e-01 -5.5666e-01  1e-03  2e-17  5e-16
 8: -5.5604e-01 -5.5621e-01  2e-04  2e-17  4e-16
 9: -5.5612e-01 -5.5612e-01  6e-06  2e-17  4e-16
10: -5.5612e-01 -5.5612e-01  7e-08  2e-17  6e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -5.9953e+00 

[ 1. -1.  1.  1. -1.  1.  1. -1.  1. -1. -1.  1. -1. -1. -1. -1.  1.  1.
 -1.  1.  1.  1.]
[ 1. -1.  1.  1. -1.  1.  1. -1.  1. -1. -1.  1. -1. -1. -1. -1.  1.  1.
 -1.  1.  1.  1.]
[ 1. -1.  1.  1. -1.  1.  1. -1.  1. -1. -1.  1. -1. -1. -1. -1.  1.  1.
 -1.  1.  1.  1.]
GM 1.0
22 out of 22 predictions correct
Accuracy 1.0
Errore quadratico medio:  0.0
     pcost       dcost       gap    pres   dres
 0: -3.1779e+01 -4.4739e+00  4e+02  2e+01  9e-16
 1: -2.4897e+00 -1.8791e+00  1e+01  7e-01  1e-15
 2: -6.7270e-01 -1.6268e+00  1e+00  1e-02  1e-15
 3: -6.3797e-01 -6.6378e-01  3e-02  9e-05  3e-15
 4: -6.3855e-01 -6.4390e-01  5e-03  2e-05  8e-16
 5: -6.3891e-01 -6.4016e-01  1e-03  2e-17  3e-16
 6: -6.3929e-01 -6.3946e-01  2e-04  2e-17  6e-16
 7: -6.3936e-01 -6.3941e-01  5e-05  4e-17  5e-16
 8: -6.3937e-01 -6.3940e-01  2e-05  2e-17  4e-16
 9: -6.3938e-01 -6.3939e-01  3e-06  2e-17  5e-16
10: -6.3938e-01 -6.3939e-01  7e-07  3e-17  5e-16
11: -6.3939e-01 -6.3939e-01  1e-07  3e-17  6e-16
Optimal 

     pcost       dcost       gap    pres   dres
 0: -4.2562e+01 -1.9399e+00  4e+02  2e+01  1e-15
 1: -1.0477e+00 -1.8757e+00  5e+00  2e-01  1e-15
 2: -6.5662e-01 -1.3312e+00  7e-01  8e-17  8e-16
 3: -6.7975e-01 -7.0490e-01  3e-02  7e-17  6e-16
 4: -6.7998e-01 -6.8027e-01  3e-04  2e-17  6e-16
 5: -6.7999e-01 -6.8002e-01  3e-05  2e-17  5e-16
 6: -6.7999e-01 -6.8000e-01  9e-06  4e-17  8e-16
 7: -6.7999e-01 -6.7999e-01  2e-06  2e-17  6e-16
 8: -6.7999e-01 -6.7999e-01  4e-07  2e-17  5e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -4.2562e+01 -1.9399e+00  4e+02  2e+01  1e-15
 1: -1.0477e+00 -1.8757e+00  5e+00  2e-01  1e-15
 2: -6.5662e-01 -1.3312e+00  7e-01  8e-17  8e-16
 3: -6.7975e-01 -7.0490e-01  3e-02  7e-17  6e-16
 4: -6.7998e-01 -6.8027e-01  3e-04  2e-17  6e-16
 5: -6.7999e-01 -6.8002e-01  3e-05  2e-17  5e-16
 6: -6.7999e-01 -6.8000e-01  9e-06  4e-17  8e-16
 7: -6.7999e-01 -6.7999e-01  2e-06  2e-17  6e-16
 8: -6.7999e-01 -6.7999e-01  4e-07  2e-17  5e-1

[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1.]
[ 1. -1. -1. -1.  1. -1. -1. -1.  1. -1. -1. -1.  1. -1. -1. -1.  1.  1.
 -1. -1. -1.  1. -1.]
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1.]
GM 0.0
16 out of 23 predictions correct
Accuracy 0.6956521739130435
Errore quadratico medio:  1.2173913043478262
     pcost       dcost       gap    pres   dres
 0: -3.9704e+01 -1.9041e+00  5e+02  2e+01  1e-15
 1: -9.7747e-01 -1.8950e+00  6e+00  2e-01  1e-15
 2: -5.7557e-01 -1.3158e+00  7e-01  3e-04  9e-16
 3: -5.7983e-01 -5.8824e-01  8e-03  4e-06  5e-16
 4: -5.8000e-01 -5.8008e-01  8e-05  4e-08  5e-16
 5: -5.8000e-01 -5.8000e-01  8e-07  4e-10  4e-16
 6: -5.8000e-01 -5.8000e-01  1e-08  5e-12  8e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -3.9704e+01 -1.9041e+00  5e+02  2e+01  1e-15
 1: -9.7747e-01 -1.8950e+00  6e+00  2e-01  1e-15
 2: -5.7557e-01 -1.3158e+00  7e-01  3e-04

...

     pcost       dcost       gap    pres   dres
 0:  5.0814e+06 -7.7468e+08  8e+08  3e-12  8e-12
 1:  4.8233e+06 -1.9268e+07  2e+07  1e-11  8e-12
 2:  2.2222e+06 -9.4088e+06  1e+07  7e-12  6e-12
 3:  7.5341e+05 -3.3068e+06  4e+06  6e-12  4e-12
 4:  1.7195e+05 -7.0764e+05  9e+05  4e-12  3e-12
 5:  2.2777e+04 -2.3694e+05  3e+05  2e-12  2e-12
 6: -6.6088e+03 -1.8275e+05  2e+05  2e-12  2e-12
 7: -3.8307e+04 -1.1779e+05  8e+04  4e-12  3e-12
 8: -4.7804e+04 -1.0812e+05  6e+04  4e-12  3e-12
 9: -5.7108e+04 -9.1476e+04  3e+04  1e-12  5e-12
10: -6.5656e+04 -7.9856e+04  1e+04  2e-12  5e-12
11: -7.0135e+04 -7.2534e+04  2e+03  1e-11  5e-12
12: -7.1099e+04 -7.1134e+04  3e+01  1e-11  7e-12
13: -7.1111e+04 -7.1114e+04  2e+00  6e-12  7e-12
14: -7.1112e+04 -7.1113e+04  3e-01  1e-11  6e-12
15: -7.1112e+04 -7.1112e+04  9e-03  7e-12  6e-12
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  5.0814e+06 -7.7468e+08  8e+08  3e-12  8e-12
 1:  4.8233e+06 -1.9268e+07  2e+07  1e-11  8e-1

     pcost       dcost       gap    pres   dres
 0:  5.2330e+06 -8.5884e+08  9e+08  1e-11  1e-11
 1:  4.9848e+06 -2.0134e+07  3e+07  1e-11  1e-11
 2:  1.9937e+06 -7.9753e+06  1e+07  3e-11  9e-12
 3:  6.8300e+05 -3.0357e+06  4e+06  1e-12  7e-12
 4:  1.4694e+05 -6.8496e+05  8e+05  3e-12  5e-12
 5:  4.1598e+03 -2.6185e+05  3e+05  8e-12  4e-12
 6: -1.8701e+04 -2.2956e+05  2e+05  4e-12  3e-12
 7: -5.9450e+04 -1.2983e+05  7e+04  2e-16  4e-12
 8: -7.6954e+04 -1.1793e+05  4e+04  2e-16  5e-12
 9: -8.5916e+04 -1.0400e+05  2e+04  2e-12  5e-12
10: -8.6690e+04 -1.0225e+05  2e+04  1e-11  5e-12
11: -9.0834e+04 -9.6533e+04  6e+03  4e-12  7e-12
12: -9.1712e+04 -9.4530e+04  3e+03  2e-12  7e-12
13: -9.2966e+04 -9.3158e+04  2e+02  7e-12  7e-12
14: -9.3055e+04 -9.3059e+04  4e+00  1e-11  7e-12
15: -9.3057e+04 -9.3057e+04  5e-01  1e-11  7e-12
16: -9.3057e+04 -9.3057e+04  3e-02  7e-12  8e-12
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  5.2330e+06 -8.5884e+08  9e+08  1e-11  1e-1

     pcost       dcost       gap    pres   dres
 0: -3.6613e+05 -8.3288e+08  8e+08  2e-12  1e-11
 1: -3.6627e+05 -9.0290e+06  9e+06  2e-11  1e-11
 2: -3.7956e+05 -7.9289e+05  4e+05  2e-11  1e-11
 3: -4.1973e+05 -7.8726e+05  4e+05  1e-11  1e-11
 4: -4.2992e+05 -7.9010e+05  4e+05  5e-12  1e-11
 5: -4.9027e+05 -7.4503e+05  3e+05  2e-11  1e-11
 6: -5.1460e+05 -7.3449e+05  2e+05  1e-11  1e-11
 7: -5.7649e+05 -6.2092e+05  4e+04  3e-11  2e-11
 8: -5.8839e+05 -6.0740e+05  2e+04  3e-11  2e-11
 9: -5.9494e+05 -6.0045e+05  6e+03  8e-11  1e-11
10: -5.9743e+05 -5.9797e+05  5e+02  8e-11  2e-11
11: -5.9769e+05 -5.9769e+05  6e+00  7e-11  2e-11
12: -5.9769e+05 -5.9769e+05  6e-02  3e-11  2e-11
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -3.6613e+05 -8.3288e+08  8e+08  2e-12  1e-11
 1: -3.6627e+05 -9.0290e+06  9e+06  2e-11  1e-11
 2: -3.7956e+05 -7.9289e+05  4e+05  2e-11  1e-11
 3: -4.1973e+05 -7.8726e+05  4e+05  1e-11  1e-11
 4: -4.2992e+05 -7.9010e+05  4e+05  5e-12  1e-1

     pcost       dcost       gap    pres   dres
 0: -4.2258e+05 -8.3299e+08  8e+08  3e-12  1e-11
 1: -4.2284e+05 -9.0036e+06  9e+06  1e-11  1e-11
 2: -4.4641e+05 -7.6385e+05  3e+05  1e-11  1e-11
 3: -6.7620e+05 -6.9924e+05  2e+04  4e-11  2e-11
 4: -6.7840e+05 -6.8104e+05  3e+03  2e-11  2e-11
 5: -6.7889e+05 -6.7971e+05  8e+02  1e-11  2e-11
 6: -6.7903e+05 -6.7941e+05  4e+02  4e-12  2e-11
 7: -6.7909e+05 -6.7930e+05  2e+02  2e-16  2e-11
 8: -6.7914e+05 -6.7923e+05  1e+02  2e-11  2e-11
 9: -6.7916e+05 -6.7920e+05  5e+01  3e-11  2e-11
10: -6.7917e+05 -6.7919e+05  2e+01  3e-11  2e-11
11: -6.7918e+05 -6.7918e+05  6e-01  7e-11  2e-11
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -4.2258e+05 -8.3299e+08  8e+08  3e-12  1e-11
 1: -4.2284e+05 -9.0036e+06  9e+06  1e-11  1e-11
 2: -4.4641e+05 -7.6385e+05  3e+05  1e-11  1e-11
 3: -6.7620e+05 -6.9924e+05  2e+04  4e-11  2e-11
 4: -6.7840e+05 -6.8104e+05  3e+03  2e-11  2e-11
 5: -6.7889e+05 -6.7971e+05  8e+02  1e-11  2e-1

     pcost       dcost       gap    pres   dres
 0:  1.5717e+02 -3.3732e+03  6e+03  3e-01  3e-15
 1:  1.3039e+02 -4.1533e+02  6e+02  1e-02  4e-15
 2:  1.6047e+01 -9.4669e+01  1e+02  2e-03  3e-15
 3: -9.4521e+00 -4.8722e+01  4e+01  4e-04  2e-15
 4: -2.0088e+01 -4.2843e+01  2e+01  1e-04  2e-15
 5: -2.0626e+01 -3.8883e+01  2e+01  3e-05  2e-15
 6: -2.6454e+01 -3.1215e+01  5e+00  2e-15  2e-15
 7: -2.7974e+01 -2.8655e+01  7e-01  1e-15  2e-15
 8: -2.8257e+01 -2.8269e+01  1e-02  2e-16  2e-15
 9: -2.8262e+01 -2.8263e+01  1e-04  2e-15  2e-15
10: -2.8262e+01 -2.8262e+01  1e-06  3e-15  2e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  1.5717e+02 -3.3732e+03  6e+03  3e-01  3e-15
 1:  1.3039e+02 -4.1533e+02  6e+02  1e-02  4e-15
 2:  1.6047e+01 -9.4669e+01  1e+02  2e-03  3e-15
 3: -9.4521e+00 -4.8722e+01  4e+01  4e-04  2e-15
 4: -2.0088e+01 -4.2843e+01  2e+01  1e-04  2e-15
 5: -2.0626e+01 -3.8883e+01  2e+01  3e-05  2e-15
 6: -2.6454e+01 -3.1215e+01  5e+00  2e-15  2e-1

In [19]:
clf.score(xTest, yTest)


[ 1. -1. -1.  1. -1. -1. -1.  1. -1.  1. -1. -1. -1. -1. -1. -1. -1.  1.
 -1. -1. -1. -1.  1. -1. -1. -1. -1.  1. -1. -1. -1. -1.  1. -1. -1.  1.
 -1. -1.]
[ 1. -1. -1.  1. -1. -1. -1.  1. -1.  1. -1. -1. -1. -1. -1. -1. -1.  1.
 -1. -1. -1. -1.  1. -1. -1. -1. -1.  1. -1. -1. -1. -1.  1. -1. -1.  1.
 -1. -1.]
[ 1. -1. -1.  1. -1. -1. -1.  1. -1.  1. -1. -1. -1. -1. -1. -1. -1.  1.
 -1. -1. -1. -1.  1. -1. -1. -1. -1.  1. -1. -1. -1. -1.  1. -1. -1.  1.
 -1. -1.]
GM 1.0
38 out of 38 predictions correct
Accuracy 1.0
Errore quadratico medio:  0.0


1.0

In [20]:
clf.best_params_

{'C': 10.0, 'sigma': 0.9}

In [21]:
best_params = clf.best_params_
best_C = best_params['C']
best_sigma = best_params['sigma']
(best_C, best_sigma)

(10.0, 0.9)

In [46]:
if __name__ == "__main__":
    import pylab as pl           
    def hyp_svm():
        
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)
        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            typ = 2            
            best_model = HYP_SVM(C=best_C, kernel=gaussian_kernel, sigma=best_sigma)  
            
            best_model.fit(X_train, y_train)
            best_model.score(X_test, y_test)            
            
    hyp_svm()

     pcost       dcost       gap    pres   dres
 0: -7.5487e+00 -2.1155e+01  5e+02  2e+01  7e-16
 1: -3.4718e+00 -1.9578e+01  2e+01  3e-01  7e-16
 2: -3.4729e+00 -5.2535e+00  2e+00  5e-03  4e-16
 3: -3.8630e+00 -4.4904e+00  6e-01  1e-03  4e-16
 4: -3.9958e+00 -4.2781e+00  3e-01  5e-04  4e-16
 5: -4.0560e+00 -4.1769e+00  1e-01  2e-04  4e-16
 6: -4.0940e+00 -4.1157e+00  2e-02  8e-06  4e-16
 7: -4.1026e+00 -4.1047e+00  2e-03  1e-15  4e-16
 8: -4.1035e+00 -4.1036e+00  1e-04  4e-16  4e-16
 9: -4.1036e+00 -4.1036e+00  2e-06  7e-16  4e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -7.5487e+00 -2.1155e+01  5e+02  2e+01  7e-16
 1: -3.4718e+00 -1.9578e+01  2e+01  3e-01  7e-16
 2: -3.4729e+00 -5.2535e+00  2e+00  5e-03  4e-16
 3: -3.8630e+00 -4.4904e+00  6e-01  1e-03  4e-16
 4: -3.9958e+00 -4.2781e+00  3e-01  5e-04  4e-16
 5: -4.0560e+00 -4.1769e+00  1e-01  2e-04  4e-16
 6: -4.0940e+00 -4.1157e+00  2e-02  8e-06  4e-16
 7: -4.1026e+00 -4.1047e+00  2e-03  1e-15  4e-1

In [ ]:
#clf_svm = svm.SVC(kernel='rbf', gamma=0.001, C=100)
#clf_svm.fit(X_train, y_train)
#y_pred_svm = clf_svm.predict(X_test) 
#acc_svm = accuracy_score(y_test, y_pred_svm)
#print ("Overall RBF KERNEL SVM accuracy: ",acc_svm)

In [ ]:
from cvxopt import matrix
class SVM(object):

    def __init__(self, kernel=gaussian_kernel, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)
    def fit(self, X, y):
        self.kernel = gaussian_kernel
        n_samples, n_features = X.shape
        # Gram matrix
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K[i,j] = gaussian_kernel(X[i], X[j])
               # print(K[i,j])
        print(K.shape)

        P = cvxopt.matrix(np.outer(y,y) * K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
       # print(a)
        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X[sv]
        self.sv_y = y[sv]
        print("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * K[ind[n],sv])
        self.b /= len(self.a)

        # Weight vector
        if self.kernel == gaussian_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
                #print(self.w)
        else:
            self.w = None

    def project(self, X):
        if self.w is None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            X=np.asarray(X)
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * gaussian_kernel(X[i], sv)
                y_predict[i] = s
              #  print(y_predict[i])
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X))

In [ ]:
if __name__ == "__main__":
    import pylab as pl           
    def normal_svm():
        
        clf = SVM(C=100.0)
        
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
        for train_index, test_index in skf.split(X, y):
            #print("TRAIN:", train_index, "\nTEST:", test_index)
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            clf.fit(X_train, y_train)    
            y_predict = clf.predict(X_test)
            gm(y_predict,y_test)
            correct = np.sum(y_predict == y_test)
            mse = mean_squared_error(y_test, y_predict)
            print("%d out of %d predictions correct" % (correct, len(y_predict)))
            print("Accuracy",correct/len(y_predict))
            print("Errore quadratico medio: ", mse)
    normal_svm()   